# SDSC3001 - Assignment 1

## Question 5

In [1]:
def load_graph(file_path):
    edges = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('#'):
                continue  # Skip comment lines
            parts = line.strip().split()
            if len(parts) == 2:
                from_node, to_node = map(int, parts)
                edges.append((from_node, to_node))
    return edges

In [ ]:
file_path = 'com-dblp.txt'
graph_edges = load_graph(file_path)
print(graph_edges)